In [1]:
from unet import UNet
import torch

unet_model_path = "unet_model.pth"

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(unet_model_path))
unet_model.eval()


/tmp/ipykernel_1205167/2283401048.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet_model.load_state_dict(torch.load(unet_model_path))


UNet(
  (enc1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (enc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_r

In [2]:
from dataset import RESNETDataset
from torch.utils.data import DataLoader

normal_dir = '../chest_xray/train/NORMAL'
pneumonia_dir = '../chest_xray/train/PNEUMONIA'

train_dataset = RESNETDataset(unet_model, device, normal_dir, pneumonia_dir, target_size=(256, 256))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [3]:
from resnet import ResNetCNN
import torch.optim as optim
import torch.nn as nn

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetCNN().to(device)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 모델 저장 경로
model_save_path = "resnet_model.pth"

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.


In [4]:
# 모델 학습
min = 999
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images.repeat(1, 3, 1, 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.5f}")
    if min > epoch_loss:
        torch.save(model.state_dict(), model_save_path)
        print(f"Model {epoch + 1} saved to {model_save_path}")
        min = epoch_loss


# 학습된 모델 저장
#torch.save(model.state_dict(), model_save_path)
#print(f"Model saved to {model_save_path}")


Epoch 1/30, Loss: 47.87099
Model 1 saved to resnet_model.pth
Epoch 2/30, Loss: 22.44897
Model 2 saved to resnet_model.pth
Epoch 3/30, Loss: 16.43862
Model 3 saved to resnet_model.pth
Epoch 4/30, Loss: 12.99708
Model 4 saved to resnet_model.pth
Epoch 5/30, Loss: 8.13874
Model 5 saved to resnet_model.pth
Epoch 6/30, Loss: 4.64698
Model 6 saved to resnet_model.pth
Epoch 7/30, Loss: 7.96268
Epoch 8/30, Loss: 4.10742
Model 8 saved to resnet_model.pth
Epoch 9/30, Loss: 3.13135
Model 9 saved to resnet_model.pth
Epoch 10/30, Loss: 2.57380
Model 10 saved to resnet_model.pth
Epoch 11/30, Loss: 6.67355
Epoch 12/30, Loss: 3.31020
Epoch 13/30, Loss: 2.51941
Model 13 saved to resnet_model.pth
Epoch 14/30, Loss: 2.61249
Epoch 15/30, Loss: 5.48285
Epoch 16/30, Loss: 1.51200
Model 16 saved to resnet_model.pth
Epoch 17/30, Loss: 0.51173
Model 17 saved to resnet_model.pth
Epoch 18/30, Loss: 1.02467
Epoch 19/30, Loss: 0.17930
Model 19 saved to resnet_model.pth
Epoch 20/30, Loss: 0.06144
Model 20 saved to 